## Federated Analysis

Here we take the weights from each of the models, aggregate them and then use these with each of the studies


In [1]:
import numpy as np

def aggregate_weights_median(weights_dict):
    """
    Aggregate weights layer-wise across all studies using the median.
    """
    studies = list(weights_dict.keys())
    n_layers = len([w for w in weights_dict[studies[0]] if w is not None])

    aggregated_weights = []
    for layer_idx in range(n_layers):
        # Collect all weights for this layer across studies
        layer_weights = [weights_dict[study][layer_idx] for study in studies if weights_dict[study] is not None]

        # Stack and take median across the first axis (studies)
        stacked = np.stack(layer_weights, axis=0)
        median_weight = np.median(stacked, axis=0)
        aggregated_weights.append(median_weight)

    return aggregated_weights


In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import os

def build_model_from_file(model_path):
    """
    Load a Keras model structure from an existing model file (without weights).
    """
    model = load_model(model_path)
    model_json = model.to_json()
    new_model = model_from_json(model_json)
    return new_model


In [3]:
def build_federated_model(aggregated_weights, reference_model_path):
    """
    Build a model with aggregated weights.
    """
    model = build_model_from_file(reference_model_path)
    model.set_weights(aggregated_weights)
    return model


In [4]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def evaluate_federated_model(model, X_test, y_true, antibody_labels):
    """
    Evaluate reconstructed output from federated model.
    """
    reconstructed = model.predict(X_test)
    mse_per_ab = {}

    for i, ab in enumerate(antibody_labels):
        mse = mean_squared_error(y_true[:, i], reconstructed[:, i])
        mse_per_ab[ab] = mse

    mse_median = np.median(list(mse_per_ab.values()))
    mse_iqr = np.subtract(*np.percentile(list(mse_per_ab.values()), [75, 25]))

    print(f"Federated Model MSE (median): {mse_median:.4f}")
    print(f"IQR: {mse_iqr:.4f}")
    print("Per-Autoantibody MSE:")
    print(pd.Series(mse_per_ab).sort_values())

    return {
        "mse_median": mse_median,
        "mse_iqr": mse_iqr,
        "per_antibody_mse": mse_per_ab,
        "reconstructed": reconstructed
    }


In [5]:
from tensorflow.keras.models import load_model

def extract_model_weights_per_study(study_list, model_dir="models"):
    weights_dict = {}

    for study in study_list:
        model_path = f"{model_dir}/{study}_model.keras"
        try:
            model = load_model(model_path)
            weights = model.get_weights()
            weights_dict[study] = weights
        except Exception as e:
            print(f"Could not load model for {study}: {e}")
            weights_dict[study] = None

    return weights_dict



In [6]:
import os
# ========== STEP 0: Set working directory (for running locally on laptop) =========
os.getcwd()
os.chdir("/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/oadr-autoantibody")
os.getcwd()

'/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/oadr-autoantibody'

In [7]:
# === Run this
study_list = ["SDY569", "SDY1625", "SDY524", "SDY797", "SDY1737"]
weights_dict = extract_model_weights_per_study(study_list)


In [9]:
# 1. Aggregate the weights
aggregated_weights = aggregate_weights_median(weights_dict)

# 2. Build federated model using any of the saved models as architecture reference
reference_model_path = "models/SDY569_model.keras"  # or any other
federated_model = build_federated_model(aggregated_weights, reference_model_path)



In [10]:
import pickle

# Load mse_summaries from file
with open("mse_summaries.pkl", "rb") as f:
    mse_summaries = pickle.load(f)

# Confirm structure
print(mse_summaries.keys())


dict_keys(['SDY569', 'SDY1625', 'SDY524', 'SDY797', 'SDY1737'])


In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Choose study
study = "SDY569"

# Get test data from mse_summaries
X_test = mse_summaries[study]["X_test"]
antibodies = mse_summaries[study]["antibody_labels"]

# Predict with federated model
reconstructed = federated_model.predict(X_test)

# Compute overall MSE
mse = mean_squared_error(X_test, reconstructed)
print(f"[Federated] MSE on {study}: {mse:.4f}")

# Per-antibody MSE
per_ab_mse = {}
for i, ab in enumerate(antibodies):
    per_ab_mse[ab] = mean_squared_error(X_test[:, i], reconstructed[:, i])

print("\n[Federated] Per-Antibody MSE:")
for ab, err in per_ab_mse.items():
    print(f"{ab}: {err:.4f}")


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(2, 7), dtype=float32). Expected shape (None, 7, 1, 1), but input has incompatible shape (2, 7)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(2, 7), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>